In [1]:
using JuMP
using Gurobi
using Combinatorics
using ToeplitzMatrices
using LinearAlgebra

In [2]:
# Define parameters of the problem
A=[0.2 0; 0 1.3]
C=[0.5 1.5; 0 1]

n_y,n_x=size(C)

H_w=[1 0; -1 0; 0 1; 0 -1]
h_w=1.5*[1; 1; 1; 1]
n_w=size(h_w,1)


H_v=[1 0; -1 0; 0 1; 0 -1]
h_v=0.3*[1; 1; 1; 1]
n_v=size(h_v,1)

H_x0=[1 0; -1 0; 0 1; 0 -1]
h_x0=1.7*[1; 1; 1; 1]
n_x0=size(h_x0,1)

T=10
N=3

bigM=1e5

100000.0

In [3]:
time_instants=0:T-1 # MODIFIED
all_schedules=collect(combinations(time_instants,N)); # do we need to collect ?
n_schedules=size(all_schedules,1)

120

In [4]:
# compute the J matrix
J=I # Identity matrix
for i = 1:T
    J = [J; (A^i)] # not efficient
end

In [5]:
# compute the S matrix
S=zeros(n_x*(T+1),n_x*T)
l=one(A) # Is it efficient, command "I" is another option...
S[n_x+1:2*n_x,1:n_x]=l # A^0
for i=1:T-1 # loop over the lines
    l=[A*l[1:n_x,1:n_x] l] # A*l[1:nx,1:nx] is A^i
    S[(i+1)*n_x+1:(i+2)*n_x, 1:(i+1)*n_x]=l
end
print("We don't use the triangular structure...\n")

We don't use the triangular structure...


In [6]:
# function to compute C_M
print("We don't use the sparse structure...\n")
function computeC_M(T,M)
    # construct C_sched (=C_bar)
    M=M.+1 # to have indices
    diag=zeros(T)
    diag[M].=1
    dMat=Diagonal(diag)
    C_sched=kron(dMat,C)
    return C_sched
end

We don't use the sparse structure...


computeC_M (generic function with 1 method)

In [7]:
# compute C_bar
print("We don't use the sparse structure...\n")

# preallocation
C_bar=zeros(n_schedules, T*n_y, (T+1)*n_x)

for schedule_idx=1:n_schedules
    M_i=all_schedules[schedule_idx]
    C_bar[schedule_idx,:,1:T*n_x].=computeC_M(T,M_i)
end

We don't use the sparse structure...


In [8]:
# compute H_eta and h_eta
H_eta=[kron(Matrix(1I,T,T), H_w) zeros(T*n_w,T*n_y + n_x) ;
    zeros(T*n_v, T*n_x) kron(Matrix(1I,T,T), H_v) zeros(T*n_v,n_x);
    zeros(n_x0,T*(n_x+n_y)) H_x0]

h_eta=[kron(ones(T),h_w); kron(ones(T),h_v); h_x0];

In [9]:
model = Model(Gurobi.Optimizer)


--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only


A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Gurobi

In [10]:
# Declare variables

@variable(model,Q[1:n_schedules,1:n_x*T,1:n_x*T])
@variable(model,r[1:n_schedules,1:n_x*T])
@variable(model,Lambda[1:n_schedules,1:2*n_x*(T+1),1:T*(n_w+n_v)+n_x0])
@variable(model,alpha0[1:T+1])
@variable(model,s[1:n_schedules],Bin);

In [11]:
alpha_bar=kron(alpha0,ones(n_x))
alpha_bar=[alpha_bar; alpha_bar];

In [12]:
# add constraints

@constraint(model,alpha0.>=0)
@constraint(model,sum(s)==1) # should be replaced by indicator constraint
@constraint(model,Lambda.>=zeros(size(Lambda)))

for schedule_idx=1:n_schedules
    P_xi_w = S + S*Q[schedule_idx,:,:]*C_bar[schedule_idx,:,:]*S
    P_xi_v = S*Q[schedule_idx,:,:]
    P_xi_xi0 = ( I + S*Q[schedule_idx,:,:]*C_bar[schedule_idx,:,:] )*J

    R_T_mT = [ I ; -Matrix(1I, n_x*(T+1), n_x*(T+1)) ]

    # Add dual equality constraint
    LHS1 = Lambda[schedule_idx,:,:] * H_eta
    RHS1 = R_T_mT * [ P_xi_w P_xi_v P_xi_xi0 ]
    @constraint(model, LHS1 .== RHS1 )
    
    # Add dual inequality constraint
    LHS2 = Lambda[schedule_idx,:,:] * h_eta    
    RHS2 = alpha_bar + bigM*(1-s[schedule_idx])*ones(size(alpha_bar)) - R_T_mT * S*r[schedule_idx,:]
    @constraint(model, LHS2.<=RHS2 )
    
    # low diagonal constraint
    Q_in = Q[schedule_idx,:,:]
    for blk_row_idx = 1:div(size(Q_in,1), n_x)-1
        LHS_bloc = Q_in[(blk_row_idx-1)*n_x.+(1:n_x),blk_row_idx*n_y+1:end]
        @constraint(model,LHS_bloc.==zeros(size(LHS_bloc)))
    end
end

In [13]:
@objective(model, Min, sum(alpha0))

alpha0[1] + alpha0[2] + alpha0[3] + alpha0[4] + alpha0[5] + alpha0[6] + alpha0[7] + alpha0[8] + alpha0[9] + alpha0[10] + alpha0[11]

In [14]:
time_beginning=time_ns()

optimize!(model)

time_ending=time_ns()
exec_time=(time_ending-time_beginning)/1e9
print("\n\n=========== exec_time = $(exec_time) seconds ===========\n\n")


--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 692172 rows, 494051 columns and 3224051 nonzeros
Model fingerprint: 0xbb000b04
Variable types: 493931 continuous, 120 integer (120 binary)
Coefficient statistics:
  Matrix range     [1e-13, 1e+05]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-07, 1e+05]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 614003 rows and 326592 columns (presolve time = 5s) ...
Presolve removed 635796 rows and 378288 columns
Presolve time: 9.65s
Presolved: 56376 rows, 115763 columns, 656963 nonzeros
Variable types: 115642 continuous, 121 integer (121 binary)

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...


Root simplex log...

Iterati

In [15]:
print("termination_status = $(termination_status(model))\n")
print("primal_status = $(primal_status(model))\n")
print("dual_status = $(dual_status(model))\n")

val_obj=objective_value(model)

idx_opt=findall(ones(size(s)).==value.(s))

schedule_opt=all_schedules[idx_opt][1] # why do I need the "[1]" ?

print("val_obj = $val_obj\n")
print("schedule_opt = $(schedule_opt)")

termination_status = OPTIMAL
primal_status = FEASIBLE_POINT
dual_status = NO_SOLUTION
val_obj = 42.562199999988245
schedule_opt = [2, 4, 7]